In [20]:
# Flask implementation

In [21]:
!pip install unidecode
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import pandas as pd

movies = pd.read_csv('imdb_top_1000.csv')

In [23]:
from langdetect import detect

def df_detect(word):
  try:
    return detect(word)
  except:
    return 'nu'

movie_language = movies.Series_Title.apply(df_detect)

In [24]:
import nltk

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

True

In [25]:
def min_max_scaler(number_list):
  min, max = number_list.min(), number_list.max()
  diff = max-min
  result = []
  for i in number_list:
    result.append((i-min)/diff)
  return result

In [33]:
from unidecode import unidecode
import nltk
import string
from sklearn.preprocessing import MinMaxScaler

stopwords = set(nltk.corpus.stopwords.words('english'))

def series_title_tokenize(sentence):
  sentence = nltk.word_tokenize(unidecode(sentence.lower()))
  return [word for word in sentence if word not in stopwords and word not in string.punctuation]

def nltk_pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return nltk.corpus.wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return nltk.corpus.wordnet.VERB
    elif nltk_tag.startswith('N'):
        return nltk.corpus.wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return nltk.corpus.wordnet.ADV
    else:
        return None

def overview_tokenize(sentence):
  sentence = nltk.word_tokenize(sentence)
  tags = [(unidecode(i[0].lower()),nltk_pos_tagger(i[1])) for i in nltk.pos_tag(sentence)]
  tags = [i for i in tags if i[0] not in stopwords and i[0] not in string.punctuation]
  lemmatizer = nltk.stem.WordNetLemmatizer()
  result = []
  for i in tags:
    if i[1]:
      result.append(lemmatizer.lemmatize(i[0], i[1]))
    else:
      result.append(i[0])
  return result

certificate_dict = {'U':0,'PG':0.2,'12A':0.4,'12':0.6,'15':0.8,'18':1}
movies_search = movies.copy()
movies_search.Series_Title = movies_search.Series_Title.apply(series_title_tokenize)
movies_search.Overview = movies_search.Overview.apply(overview_tokenize)
#movies_search.Released_Year = min_max_scaler(movies_search.Released_Year)
movies_search.Runtime = movies_search.Runtime.apply(lambda x: int(x.replace(' min', '')))
movies_search.Runtime = min_max_scaler(movies_search.Runtime)
#movies_search.Certificate = movies_search.Certificate.replace(certificate_dict)
movies_search.Genre = movies_search.Genre.apply(lambda x: set(x.split(', ')))
#movies_search.insert(2,'Language',movie_language)
movies_search.insert(11,'Stars',movies_search.apply(lambda x: {x.Star1,x.Star2,x.Star3,x.Star4}, axis=1))
movies_search = movies_search.drop(['Poster_Link','Meta_score','Gross','Star1','Star2','Star3','Star4','IMDB_Rating','No_of_Votes'],axis=1)

In [34]:
# Visualise 

movies_search

,Series_Title,Released_Year,Certificate,Runtime,Genre,Overview,Director,Stars
0,"[shawshank, redemption]",1994,A,0.351449,{Drama},"[two, imprisoned, men, bond, number, year, fin...",Frank Darabont,"{Tim Robbins, Bob Gunton, William Sadler, Morg..."
1,[godfather],1972,A,0.471014,"{Drama, Crime}","[organize, crime, dynasty, 's, age, patriarch,...",Francis Ford Coppola,"{Marlon Brando, Diane Keaton, James Caan, Al P..."
2,"[dark, knight]",2008,UA,0.387681,"{Drama, Action, Crime}","[menace, know, joker, wreak, havoc, chaos, peo...",Christopher Nolan,"{Michael Caine, Aaron Eckhart, Heath Ledger, C..."
3,"[godfather, part, ii]",1974,A,0.568841,"{Drama, Crime}","[early, life, career, vito, corleone, 1920s, n...",Francis Ford Coppola,"{Robert Duvall, Diane Keaton, Robert De Niro, ..."
4,"[12, angry, men]",1957,U,0.184783,"{Drama, Crime}","[jury, holdout, attempt, prevent, miscarriage,...",Sidney Lumet,"{Henry Fonda, John Fiedler, Lee J. Cobb, Marti..."
...,...,...,...,...,...,...,...,...
995,"[breakfast, tiffany, 's]",1961,A,0.253623,"{Comedy, Drama, Romance}","[young, new, york, socialite, becomes, interes...",Blake Edwards,"{George Peppard, Buddy Ebsen, Patricia Neal, A..."
996,[giant],1956,G,0.565217,"{Drama, Western}","[sprawl, epic, cover, life, texas, cattle, ran...",George Stevens,"{James Dean, Carroll Baker, Rock Hudson, Eliza..."
997,[eternity],1953,Passed,0.264493,"{Drama, Romance, War}","[hawaii, 1941, private, cruelly, punish, box, ...",Fred Zinnemann,"{Montgomery Clift, Burt Lancaster, Donna Reed,..."
998,[lifeboat],1944,NaN,0.188406,"{Drama, War}","[several, survivor, torpedoed, merchant, ship,...",Alfred Hitchcock,"{John Hodiak, Walter Slezak, William Bendix, T..."


In [35]:
import numpy as np
from numpy.linalg import norm

def cosine_similarity(x,y):
  return np.dot(x,y)/(norm(x)*norm(y))

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer=lambda x: x)
overview_vectorized = vectorizer.fit_transform(movies_search.Overview)
tfidf_overview = pd.DataFrame(overview_vectorized.toarray())
tfidf_overview = tfidf_overview.astype(str).agg(', '.join, axis=1)
tfidf_overview = tfidf_overview.apply(lambda x: list(map(float, list(x.split(', ')))))
movies_search.Overview = tfidf_overview

series_title_vectorized = vectorizer.fit_transform(movies_search.Series_Title)
tfidf_series_title = pd.DataFrame(series_title_vectorized.toarray())
tfidf_series_title = tfidf_series_title.astype(str).agg(', '.join, axis=1)
tfidf_series_title = tfidf_series_title.apply(lambda x: list(map(float, list(x.split(', ')))))
movies_search.Series_Title = tfidf_series_title

In [43]:
def number_change(number1,number2):
  return 1-abs(number1-number2)

def director_match(query,document):
  if query == document:
    return 1
  return 0

def stars_match(query,document):
  n = 0
  for i in query:
    if i in document:
      n+=1
  return n/4

def genre_match(query,document):
  n = 0
  length = len(query)
  for i in query:
    if i in document:
      n+=1
  return n/length

def find_titles(query):
  query = series_title_tokenize(query)
  query = vectorizer.transform([query]).todense()
  titles = movies_search.Series_Title.apply(lambda x: cosine_similarity(query,x)[0,0])
  title = titles.idxmax()
  return movies_search.iloc[title]

def evaluate(query,row):
  title_score = cosine_similarity(query.Series_Title,row.Series_Title)
  #year_score = number_change(query.Released_Year,row.Released_Year)
  #age_score = number_change(query.Certificate,row.Certificate)
  time_score = number_change(query.Runtime,row.Runtime)
  genre_score = genre_match(query.Genre,row.Genre)
  overview_score = cosine_similarity(query.Overview,row.Overview)
  director_score = director_match(query.Director,row.Director)
  stars_score = stars_match(query.Stars,row.Stars)
  #return title_score+year_score+age_score+time_score+genre_score+overview_score+director_score+stars_score
  return title_score+time_score+genre_score+overview_score+director_score+stars_score
def search(query):
  try:
    query = find_titles(query)
    search_index = movies_search.apply(lambda x: evaluate(query,x),axis=1).sort_values(ascending=False).index.values
    return movies.iloc[search_index]
  except:
    return 'No matches'

In [44]:
query = movies_search.iloc[0]
search_index = movies_search.apply(lambda x: evaluate(query,x),axis=1).sort_values(ascending=False).index.values
movies.iloc[search_index]

<ipython-input-35-1313b1598f13>:5: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(x,y)/(norm(x)*norm(y))


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
25,https://m.media-amazon.com/images/M/MV5BMTUxMz...,The Green Mile,1999,A,189 min,"Crime, Drama, Fantasy",8.6,The lives of guards on Death Row are affected ...,61.0,Frank Darabont,Tom Hanks,Michael Clarke Duncan,David Morse,Bonnie Hunt,1147794,"136,801,374"
505,https://m.media-amazon.com/images/M/MV5BMTIzND...,Mystic River,2003,A,138 min,"Crime, Drama, Mystery",7.9,The lives of three men who were childhood frie...,84.0,Clint Eastwood,Sean Penn,Tim Robbins,Kevin Bacon,Emmy Rossum,419420,"90,135,191"
27,https://m.media-amazon.com/images/M/MV5BOTUwOD...,Se7en,1995,A,127 min,"Crime, Drama, Mystery",8.6,"Two detectives, a rookie and a veteran, hunt a...",65.0,David Fincher,Morgan Freeman,Brad Pitt,Kevin Spacey,Andrew Kevin Walker,1445096,"100,125,643"
234,https://m.media-amazon.com/images/M/MV5BMTkxNz...,Million Dollar Baby,2004,UA,132 min,"Drama, Sport",8.1,A determined woman works with a hardened boxin...,86.0,Clint Eastwood,Hilary Swank,Clint Eastwood,Morgan Freeman,Jay Baruchel,635975,"100,492,203"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,https://m.media-amazon.com/images/M/MV5BYTIwND...,To Be or Not to Be,1942,Passed,99 min,"Comedy, War",8.2,"During the Nazi occupation of Poland, an actin...",86.0,Ernst Lubitsch,Carole Lombard,Jack Benny,Robert Stack,Felix Bressart,29915,NaN
341,https://m.media-amazon.com/images/M/MV5BMjA1Nz...,Her,2013,A,126 min,"Drama, Romance, Sci-Fi",8.0,"In a near future, a lonely writer develops an ...",90.0,Spike Jonze,Joaquin Phoenix,Amy Adams,Scarlett Johansson,Rooney Mara,540772,"25,568,251"
413,https://m.media-amazon.com/images/M/MV5BNjAzNz...,Being There,1979,PG,130 min,"Comedy, Drama",8.0,"A simpleminded, sheltered gardener becomes an ...",83.0,Hal Ashby,Peter Sellers,Shirley MacLaine,Melvyn Douglas,Jack Warden,65625,"30,177,511"
617,https://m.media-amazon.com/images/M/MV5BYWUxZj...,Once,2007,R,86 min,"Drama, Music, Romance",7.8,A modern-day musical about a busker and an imm...,88.0,John Carney,Glen Hansard,Markéta Irglová,Hugh Walsh,Gerard Hendrick,110656,"9,439,923"


## Interface: Flask implementation

In [ ]:
# All your previous imports and functions go here

from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/search', methods=['GET'])
def search_movie():
    query = request.args.get('query')
    if query:
        result = search(query)
        if isinstance(result, str):
            return jsonify({'error': result})
        else:
            return jsonify(result.to_dict())
    else:
        return jsonify({'error': 'Please provide a query parameter.'})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
